In [31]:
import time
import numpy as np
import pinocchio as pin
import crocoddyl

from robot_properties_solo.config import Solo12Config
from py_biconvex_mpc.ik.inverse_kinematics import InverseKinematics
from py_biconvex_mpc.ik_utils.gait_generator import GaitGenerator

from cnt_plan_utils import SoloCntGen


In [32]:
robot = Solo12Config.buildRobotWrapper()
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=True)
viz.loadViewerModel()

n_eff = 4
m = pin.computeTotalMass(robot.model)
q0 = np.array(Solo12Config.initial_configuration)
x0 = np.concatenate([q0, pin.utils.zero(robot.model.nv)])

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [45]:
# initial and ter state
X_init = np.zeros(9)
X_init[0:3] = q0[0:3]
X_ter = X_init.copy()

# contact plan
st = 0.4 # step time 
sh = 0.1 # step height
sl = np.array([0.1,0.0,0]) # step length
n_steps = 10 # number of steps
T = st*(n_steps + 2)
dt = 1e-2

X_ter[0:3] += sl*(n_steps)

cnt_planner = SoloCntGen(T, dt, 1)
# print(cnt_plan)

In [56]:
cnt_planner.reset(T, dt)
cnt_plan = cnt_planner.create_trot_plan(st, sl, n_steps)
ik_solver = cnt_planner.create_ik_step_costs(cnt_plan, sh, [1e+4, 1e+3])

In [57]:
mass = pin.computeTotalMass(robot.model, robot.data)
com_loc = pin.centerOfMass(robot.model, robot.data, q0)
mom_opt = np.zeros((int(T/dt), 6))
mom_opt[:,0] = 3*mass*sl[0]/T

In [ ]:
# ik_solver.ik.add_com_position_tracking_task(0, T, com_opt, 1e+4, "com_track_cost")
ik_solver.create_centroidal_task(mom_opt, 0, T, "mom_track_cost", 1e+1)
xs = ik_solver.optimize(x0, wt_xreg=5e-4)

In [49]:
for i in range(len(xs)):
    time.sleep(0.01)
    viz.display(xs[i][:robot.model.nq])

In [7]:
for i in range(len(xs)):
    q = xs[i][:robot.model.nq]
    v = xs[i][robot.model.nq:]
    pin.forwardKinematics(robot.model, robot.data, q, v)
    pin.updateFramePlacements(robot.model, robot.data)
#     print(pin.centerOfMass(robot.model, robot.data, q, v))
    pin.computeCentroidalMomentum(robot.model, robot.data)
#     print(np.round(robot.data.hg, 2)[3], mom_opt[i][3])
    